In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [52]:
pipe = pipeline(model="facebook/bart-large-mnli")
outputs = pipe('пожелать',
    candidate_labels=['Подарок', 'Пожелание'],
)
outputs

Device set to use mps:0


{'sequence': 'пожелать',
 'labels': ['Пожелание', 'Подарок'],
 'scores': [0.816169023513794, 0.18383094668388367]}

In [53]:
outputs['labels'][0]

'Пожелание'

In [40]:
pipe('подготовить сюрприз родителям',
    candidate_labels=['подарок', 'желание'],
)

{'sequence': 'подготовить сюрприз родителям',
 'labels': ['подарок', 'желание'],
 'scores': [0.663544237613678, 0.3364557921886444]}

In [ ]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)


def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

classes = ['подарок', 'желание']
predict_zero_shot('подготовить сюрприз родителям', classes, model, tokenizer)

array([0.37635893, 0.6236411 ], dtype=float32)

In [44]:
import yaml

with open('/Users/timmiakov/Code/AI-telegram-bot/src/config/bot_config.yaml', 'r') as f:
    data = yaml.load(f, Loader=yaml.Loader)

In [45]:
data

{'classifier_model': {'model_name': 'facebook/bart-large-mnli',
  'message_types': ['Подарок', 'Желание']}}

In [1]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="MTSAIR/Cotype-Nano")
pipe(messages)

/Users/timmiakov/Code/AI-telegram-bot/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.14k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Device set to use mps:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am an AI language model developed by OpenAI. My purpose is to assist users like you by'}]}]